In [23]:
import altair
import matplotlib.pyplot as plt
from numba import njit, int64, float64
from numba.typed import List
from numba.types import Tuple, float64
import polars as pl
import numpy as np
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest, LIMIT, GTC, NONE, NEW, FILLED, CANCELED, EXPIRED, GTX, Recorder, ROIVectorMarketDepthBacktest, BUY_EVENT

In [49]:
beerusdt = np.load('beerusdt_20240601.npz')['data']
beerusdt
# np.set_printoptions(suppress=True)

array([(2684354563, 1717199997894000000, 1717200000224723000, 0.18502,    0., 0, 0, 0.),
       (2684354564, 1717199997894000000, 1717200000224723000, 0.1862 ,  146., 0, 0, 0.),
       (2684354564, 1717199997894000000, 1717200000224723000, 0.18602,  236., 0, 0, 0.),
       ...,
       (3758096385, 1717286399754000000, 1717286399789461000, 0.1674 ,    9., 0, 0, 0.),
       (3758096385, 1717286399795000000, 1717286399830001000, 0.16759,    9., 0, 0, 0.),
       (3758096385, 1717286399814000000, 1717286399849473000, 0.1676 , 4001., 0, 0, 0.)],
      dtype=[('ev', '<u8'), ('exch_ts', '<i8'), ('local_ts', '<i8'), ('px', '<f8'), ('qty', '<f8'), ('order_id', '<u8'), ('ival', '<i8'), ('fval', '<f8')])

In [50]:
asset = (
    BacktestAsset()
        .data(beerusdt)
        .linear_asset(1.0)
        .constant_latency(10_000_000, 10_000_000)
        .risk_adverse_queue_model()
        .no_partial_fill_exchange()
        .trading_value_fee_model(-0.0005, 0.0005)
        .tick_size(0.00001)
        .lot_size(0.1)
        .last_trades_capacity(2)
        .roi_lb(0.15)
        .roi_ub(0.19)
)

In [52]:
@njit
def orderbookimbalance(hbt, out):
    roi_lb_tick = int(round(0.15 / 0.00001))
    roi_ub_tick = int(round(0.19 / 0.00001))

    while hbt.elapse(60 * 1e9) == 0:
        depth = hbt.depth(0)

        mid_price = (depth.best_bid + depth.best_ask) / 2.0

        sum_ask_qty_50bp = 0.0
        sum_ask_qty = 0.0
        for price_tick in range(depth.best_ask_tick, roi_ub_tick + 1):
            if price_tick < roi_lb_tick or price_tick > roi_ub_tick:
                continue
            t = price_tick - roi_lb_tick

            ask_price = price_tick * depth.tick_size
            depth_from_mid = (ask_price - mid_price) / mid_price
            if depth_from_mid > 0.01:
                break
            sum_ask_qty += depth.ask_depth[t]

            if depth_from_mid <= 0.005:
                sum_ask_qty_50bp = sum_ask_qty


        sum_bid_qty_50bp = 0.0
        sum_bid_qty = 0.0
        for price_tick in range(depth.best_bid_tick, roi_lb_tick - 1, -1):
            if price_tick < roi_lb_tick or price_tick > roi_ub_tick:
                continue
            t = price_tick - roi_lb_tick

            bid_price = price_tick * depth.tick_size
            depth_from_mid = (mid_price - bid_price) / mid_price
            if depth_from_mid > 0.01:
                break
            sum_bid_qty += depth.bid_depth[t]

            if depth_from_mid <= 0.005:
                sum_bid_qty_50bp = sum_bid_qty

        imbalance_50bp = sum_bid_qty_50bp - sum_ask_qty_50bp
        imbalance_1pct = sum_bid_qty - sum_ask_qty
        imbalance_tob = depth.bid_depth[depth.best_bid_tick - roi_lb_tick] - depth.ask_depth[depth.best_ask_tick - roi_lb_tick]

        # out.append((hbt.current_timestamp, depth.bid_depth[depth.best_ask_tick], depth.best_bid_tick, depth.bid_depth[depth.best_bid_tick - roi_lb_tick]))
        out.append((hbt.current_timestamp, imbalance_tob, imbalance_50bp, imbalance_1pct))

        # print(f'Mid Price: {str(mid_price)}, TOB Imbalance: {imbalance_tob}, 0.5% Imbalance: {imbalance_50bp}, 1% Imbalance: {imbalance_1pct}')
    return True

hbt = ROIVectorMarketDepthBacktest([asset])

tup_ty = Tuple((float64, float64, float64, float64))
out = List.empty_list(tup_ty)
orderbookimbalance(hbt, out)
_ = hbt.close()

df = pl.DataFrame(out).transpose()
df.columns = ['Local Timestamp', 'TOB Imbalance', '0.5% Imbalance', '1% Imbalance']
df = df.with_columns(
    pl.from_epoch('Local Timestamp', time_unit='ns')
)

df


Local Timestamp,TOB Imbalance,0.5% Imbalance,1% Imbalance
datetime[ns],f64,f64,f64
2024-06-01 00:00:57.894000128,-203.0,1419.0,-796.0
2024-06-01 00:01:57.894000128,2454.0,4759.0,7620.0
2024-06-01 00:02:57.894000128,0.0,-5850.0,-4357.0
2024-06-01 00:03:57.894000128,232.0,2100.0,1193.0
2024-06-01 00:04:57.894000128,322.0,-10971.0,-9471.0
…,…,…,…
2024-06-01 23:55:57.894000128,-97.0,19819.0,22340.0
2024-06-01 23:56:57.894000128,-95.0,6996.0,7516.0
2024-06-01 23:57:57.894000128,87.0,12595.0,13572.0


In [54]:
import hvplot.polars
hvplot.extension('bokeh')
df.hvplot(x='Local Timestamp', width=1000, height=500)


:NdOverlay   [Variable]
   :Curve   [Local Timestamp]   (value)

In [28]:
@njit
def print_last_trades(hbt):
    while hbt.elapse(60 * 1e9) == 0:
        print('-------------------------------------------------------------------------------')
        print('current_timestamp:', hbt.current_timestamp)

        # Gets the last trades occurring in the market, not the trades of our orders.
        last_trades = hbt.last_trades(0)

        num = 0
        for last_trade in last_trades:
            if num > 10:
                print('...')
                break
            print(
                'exch_timestamp:',
                last_trade.exch_ts,
                'buy' if (last_trade.ev & BUY_EVENT) == BUY_EVENT else 'sell',
                last_trade.qty,
                '@',
                last_trade.px
            )
            num += 1

        # To prevent accumulating all last trades, which may cause a slowdown,
        # clear_last_trades needs to be called.
        # After this, accessing `last_trades` will cause a crash.
        hbt.clear_last_trades(0)
    return True

hbt = ROIVectorMarketDepthBacktest([asset])
print_last_trades(hbt)
_ = hbt.close()

-------------------------------------------------------------------------------
current_timestamp: 1717200057894000000
exch_timestamp: 1717200000731000000 buy 4.0 @ 0.18625
exch_timestamp: 1717200000731000000 buy 4.0 @ 0.18625
exch_timestamp: 1717200000731000000 buy 4.0 @ 0.18625
exch_timestamp: 1717200000731000000 buy 4.0 @ 0.18625
exch_timestamp: 1717200000731000000 buy 4.0 @ 0.18625
exch_timestamp: 1717200000731000000 buy 4.0 @ 0.18625
exch_timestamp: 1717200000731000000 buy 4.0 @ 0.18625
exch_timestamp: 1717200000731000000 buy 4.0 @ 0.18627
exch_timestamp: 1717200000731000000 buy 4.0 @ 0.18627
exch_timestamp: 1717200000731000000 buy 111.0 @ 0.18627
exch_timestamp: 1717200000731000000 buy 3129.0 @ 0.18627
...
-------------------------------------------------------------------------------
current_timestamp: 1717200117894000000
exch_timestamp: 1717200059719000000 sell 236.0 @ 0.18581
exch_timestamp: 1717200059719000000 sell 435.0 @ 0.1858
exch_timestamp: 1717200059719000000 sell 236.0

In [29]:
@njit
def print_3depth(hbt):
    while hbt.elapse(60_000_000_000) == 0:
        print('current_timestamp:', hbt.current_timestamp)

        # Gets the market depth for the first asset, in the same order as when you created the backtest.
        depth = hbt.depth(0)

        # a key of bid_depth or ask_depth is price in ticks.
        # (integer) price_tick = rice / tick_size
        i = 0
        for price_tick in range(depth.best_ask_tick, depth.best_ask_tick + 100):
            qty = depth.ask_qty_at_tick(price_tick)
            if qty > 0:
                print(
                    'ask: ',
                    qty,
                    '@',
                    np.round(price_tick * depth.tick_size, 5)
                )

                i += 1
                if i == 3:
                    break
        i = 0
        for price_tick in range(depth.best_bid_tick, max(depth.best_bid_tick - 100, 0), -1):
            qty = depth.bid_qty_at_tick(price_tick)
            if qty > 0:
                print(
                    'bid: ',
                    qty,
                    '@',
                    np.round(price_tick * depth.tick_size, 5)
                )

                i += 1
                if i == 3:
                    break
    return True

hbt = ROIVectorMarketDepthBacktest([asset])
print_3depth(hbt)
_ = hbt.close()

current_timestamp: 1717200057894000000
ask:  439.0 @ 0.18612
ask:  4050.0 @ 0.18613
ask:  8.0 @ 0.18616
bid:  236.0 @ 0.18577
bid:  671.0 @ 0.18576
bid:  236.0 @ 0.18575
current_timestamp: 1717200117894000000
ask:  236.0 @ 0.18579
ask:  671.0 @ 0.1858
ask:  240.0 @ 0.18581
bid:  2690.0 @ 0.18557
bid:  7235.0 @ 0.18556
bid:  236.0 @ 0.18552
current_timestamp: 1717200177894000000
ask:  12.0 @ 0.18536
ask:  4.0 @ 0.18537
ask:  100.0 @ 0.18538
bid:  12.0 @ 0.18516
bid:  237.0 @ 0.18515
bid:  237.0 @ 0.18514
current_timestamp: 1717200237894000000
ask:  4.0 @ 0.18581
ask:  252.0 @ 0.18582
ask:  12.0 @ 0.18584
bid:  236.0 @ 0.18561
bid:  236.0 @ 0.1856
bid:  236.0 @ 0.18559
current_timestamp: 1717200297894000000
ask:  4.0 @ 0.18522
ask:  245.0 @ 0.18526
ask:  245.0 @ 0.18528
bid:  326.0 @ 0.18509
bid:  8.0 @ 0.18494
bid:  1077.0 @ 0.18493
current_timestamp: 1717200357894000000
ask:  694.0 @ 0.18536
ask:  237.0 @ 0.18537
ask:  249.0 @ 0.18539
bid:  414.0 @ 0.18509
bid:  136.0 @ 0.18508
bid:  4

In [36]:
@njit
def rolling_vwap(hbt, out):
    buy_amount_bin = np.zeros(100_000, np.float64)
    buy_qty_bin = np.zeros(100_000, np.float64)
    sell_amount_bin = np.zeros(100_000, np.float64)
    sell_qty_bin = np.zeros(100_000, np.float64)

    idx = 0
    last_trade_price = np.nan

    while hbt.elapse(10 * 1e9) == 0:
        last_trades = hbt.last_trades(0)

        for last_trade in last_trades:
            if (last_trade.ev & BUY_EVENT) == BUY_EVENT:
                buy_amount_bin[idx] += last_trade.px * last_trade.qty
                buy_qty_bin[idx] += last_trade.qty
            else:
                sell_amount_bin[idx] += last_trade.px * last_trade.qty
                sell_qty_bin[idx] += last_trade.qty

        hbt.clear_last_trades(0)
        idx += 1

        if idx >= 1:
            vwap10sec = np.divide(
                buy_amount_bin[idx - 1] + sell_amount_bin[idx - 1],
                buy_qty_bin[idx - 1] + sell_qty_bin[idx - 1]
            )
        else:
            vwap10sec = np.nan

        if idx >= 6:
            vwap1m = np.divide(
                np.sum(buy_amount_bin[idx - 6:idx]) + np.sum(sell_amount_bin[idx - 6:idx]),
                np.sum(buy_qty_bin[idx - 6:idx]) + np.sum(sell_qty_bin[idx - 6:idx])
            )
            buy_vwap1m = np.divide(np.sum(buy_amount_bin[idx - 6:idx]), np.sum(buy_qty_bin[idx - 6:idx]))
            sell_vwap1m = np.divide(np.sum(sell_amount_bin[idx - 6:idx]), np.sum(sell_qty_bin[idx - 6:idx]))
        else:
            vwap1m = np.nan
            buy_vwap1m = np.nan
            sell_vwap1m = np.nan

        out.append((hbt.current_timestamp, vwap10sec, vwap1m, buy_vwap1m, sell_vwap1m))
    return True

hbt = ROIVectorMarketDepthBacktest([asset])

tup_ty = Tuple((float64, float64, float64, float64, float64))
out = List.empty_list(tup_ty, allocated=100_000)

rolling_vwap(hbt, out)

_ = hbt.close()

df = pl.DataFrame(out).transpose()
df.columns = ['Local Timestamp', '10-sec VWAP', '1-min VWAP', '1-min Buy VWAP', '1-min Sell VWAP']
df = df.with_columns(
    pl.from_epoch('Local Timestamp', time_unit='ns')
)

df

Local Timestamp,10-sec VWAP,1-min VWAP,1-min Buy VWAP,1-min Sell VWAP
datetime[ns],f64,f64,f64,f64
2024-06-01 00:00:07.894000128,0.186267,NaN,NaN,NaN
2024-06-01 00:00:17.894000128,0.186135,NaN,NaN,NaN
2024-06-01 00:00:27.894000128,0.186143,NaN,NaN,NaN
2024-06-01 00:00:37.894000128,0.186094,NaN,NaN,NaN
2024-06-01 00:00:47.894000128,0.185902,NaN,NaN,NaN
…,…,…,…,…
2024-06-01 23:59:17.894000128,0.168581,0.168144,0.168246,0.168009
2024-06-01 23:59:27.894000128,0.168713,0.168164,0.16826,0.168035
2024-06-01 23:59:37.894000128,0.168739,0.168185,0.168264,0.168081


In [37]:
df.hvplot(x='Local Timestamp')

:NdOverlay   [Variable]
   :Curve   [Local Timestamp]   (value)